In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import validation_curve
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.preprocessing import MinMaxScaler

In [2]:
solar=pd.read_csv('SolarPrediction.csv')

In [3]:
solar.head(4)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 0:00,23:55:26,1.21,48,30.46,59,177.39,5.62,6:13:00,18:13:00
1,1475229023,9/29/2016 0:00,23:50:23,1.21,48,30.46,58,176.78,3.37,6:13:00,18:13:00
2,1475228726,9/29/2016 0:00,23:45:26,1.23,48,30.46,57,158.75,3.37,6:13:00,18:13:00
3,1475228421,9/29/2016 0:00,23:40:21,1.21,48,30.46,60,137.71,3.37,6:13:00,18:13:00


In [4]:
solar.dtypes

UNIXTime                    int64
Data                       object
Time                       object
Radiation                 float64
Temperature                 int64
Pressure                  float64
Humidity                    int64
WindDirection(Degrees)    float64
Speed                     float64
TimeSunRise                object
TimeSunSet                 object
dtype: object

In [5]:
solar.describe()

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
count,3.268600e+04,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000
mean,1.478047e+09,207.124697,51.103255,30.422879,75.016307,143.489821,6.243869
std,3.005037e+06,315.916387,6.201157,0.054673,25.990219,83.167500,3.490474
min,1.472724e+09,1.110000,34.000000,30.190000,8.000000,0.090000,0.000000
25%,1.475546e+09,1.230000,46.000000,30.400000,56.000000,82.227500,3.370000
50%,1.478026e+09,2.660000,50.000000,30.430000,85.000000,147.700000,5.620000
75%,1.480480e+09,354.235000,55.000000,30.460000,97.000000,179.310000,7.870000
max,1.483265e+09,1601.260000,71.000000,30.560000,103.000000,359.950000,40.500000


In [6]:
solar.drop('TimeSunRise', axis=1,inplace=True)
solar.drop('TimeSunSet', axis=1,inplace=True)
solar.drop('Data', axis=1,inplace=True)
solar.drop('UNIXTime', axis=1,inplace=True)

In [7]:
solar['hour']=solar.Time.str[:2]

In [8]:
solar.head(3)

,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,hour
0,23:55:26,1.21,48,30.46,59,177.39,5.62,23
1,23:50:23,1.21,48,30.46,58,176.78,3.37,23
2,23:45:26,1.23,48,30.46,57,158.75,3.37,23


In [9]:
solar.drop('Time', axis=1,inplace=True)

In [10]:
# creating features and target sets
X_solar, y_solar=solar.iloc[:,solar.columns!='Radiation'], solar['Radiation']
X_solar.head(4)

,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,hour
0,48,30.46,59,177.39,5.62,23
1,48,30.46,58,176.78,3.37,23
2,48,30.46,57,158.75,3.37,23
3,48,30.46,60,137.71,3.37,23


In [11]:
X_dummies=pd.get_dummies(X_solar)
X_dummies

,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,hour_0:,hour_10,hour_11,hour_12,hour_13,...,hour_22,hour_23,hour_2:,hour_3:,hour_4:,hour_5:,hour_6:,hour_7:,hour_8:,hour_9:
0,48,30.46,59,177.39,5.62,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,48,30.46,58,176.78,3.37,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,48,30.46,57,158.75,3.37,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,48,30.46,60,137.71,3.37,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,48,30.46,62,104.95,5.62,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,44,30.43,102,145.42,6.75,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32682,44,30.42,102,117.78,6.75,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32683,44,30.42,102,145.19,9.00,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32684,44,30.42,101,164.19,7.87,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
y_solar.head(3)

0    1.21
1    1.21
2    1.23
Name: Radiation, dtype: float64

In [13]:
X_train, X_test, y_train, y_test=train_test_split(X_dummies, y_solar, random_state=0)
knn_reg_solar=KNeighborsRegressor(n_neighbors=9)


In [14]:
knn_reg_solar.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=9)

In [15]:
print('knn_reg_solar acc on train: {:.2%}'.format(knn_reg_solar.score(X_train, y_train)))
print('knn_reg_solar acc on test: {:.2%}'.format(knn_reg_solar.score(X_test, y_test)))

knn_reg_solar acc on train: 73.91%
knn_reg_solar acc on test: 65.98%


In [16]:
lr=LinearRegression()
# fit the model
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
# intercept value
lr.intercept_

-1029160713022.2917

In [18]:
# coefficient value
lr.coef_

array([ 1.75110658e+01, -3.48622891e+02, -8.90889735e-01, -2.10965627e-01,
        2.53369972e+00,  1.02916072e+12,  1.02916072e+12,  1.02916072e+12,
        1.02916072e+12,  1.02916072e+12,  1.02916072e+12,  1.02916072e+12,
        1.02916072e+12,  1.02916072e+12,  1.02916072e+12,  1.02916072e+12,
        1.02916072e+12,  1.02916072e+12,  1.02916072e+12,  1.02916072e+12,
        1.02916072e+12,  1.02916072e+12,  1.02916072e+12,  1.02916072e+12,
        1.02916072e+12,  1.02916072e+12,  1.02916072e+12,  1.02916072e+12,
        1.02916072e+12])

In [19]:
# defining the models
knn_reg_solar2=KNeighborsRegressor(n_neighbors=2)
knn_reg_solar6=KNeighborsRegressor(n_neighbors=6)
knn_reg_solar15=KNeighborsRegressor(n_neighbors=15)
knn_reg_solar25=KNeighborsRegressor(n_neighbors=25)

# fitting the models
knn_reg_solar2.fit(X_train,y_train)
knn_reg_solar6.fit(X_train,y_train)
knn_reg_solar15.fit(X_train,y_train)
knn_reg_solar25.fit(X_train,y_train)


KNeighborsRegressor(n_neighbors=25)

In [20]:
# evaluating the models
print('knn_reg_solar acc on train: {:.2%}'.format(knn_reg_solar.score(X_train,y_train)))
print('knn_reg_solar acc on test: {:.2%}'.format(knn_reg_solar.score(X_test,y_test)))

print('knn_reg_solar2 acc on train: {:.2%}'.format(knn_reg_solar2.score(X_train,y_train)))
print('knn_reg_solar2 acc on test: {:.2%}'.format(knn_reg_solar2.score(X_test,y_test)))

print('knn_reg_solar6 acc on train: {:.2%}'.format(knn_reg_solar6.score(X_train,y_train)))
print('knn_reg_solar6 acc on test: {:.2%}'.format(knn_reg_solar6.score(X_test,y_test)))

print('knn_reg_solar15 acc on train: {:.2%}'.format(knn_reg_solar15.score(X_train,y_train)))
print('knn_reg_solar15 acc on test: {:.2%}'.format(knn_reg_solar15.score(X_test,y_test)))

print('knn_reg_solar25 acc on train: {:.2%}'.format(knn_reg_solar25.score(X_train,y_train)))
print('knn_reg_solar25 acc on test: {:.2%}'.format(knn_reg_solar25.score(X_test,y_test)))

knn_reg_solar acc on train: 73.91%
knn_reg_solar acc on test: 65.98%
knn_reg_solar2 acc on train: 86.68%
knn_reg_solar2 acc on test: 57.68%
knn_reg_solar6 acc on train: 76.27%
knn_reg_solar6 acc on test: 65.36%
knn_reg_solar15 acc on train: 71.40%
knn_reg_solar15 acc on test: 66.44%
knn_reg_solar25 acc on train: 69.19%
knn_reg_solar25 acc on test: 65.96%


In [21]:
X_solar.columns

Index(['Temperature', 'Pressure', 'Humidity', 'WindDirection(Degrees)',
       'Speed', 'hour'],
      dtype='object')

In [22]:
p1=[48,30.46,59,177.39,5.62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
lr.predict([p1])

array([26.55651855])

In [23]:
p2=[80,30.46,59,177.39,5.62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
lr.predict([p2])

array([586.91064453])

In [24]:
solar.describe()

,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
count,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000
mean,207.124697,51.103255,30.422879,75.016307,143.489821,6.243869
std,315.916387,6.201157,0.054673,25.990219,83.167500,3.490474
min,1.110000,34.000000,30.190000,8.000000,0.090000,0.000000
25%,1.230000,46.000000,30.400000,56.000000,82.227500,3.370000
50%,2.660000,50.000000,30.430000,85.000000,147.700000,5.620000
75%,354.235000,55.000000,30.460000,97.000000,179.310000,7.870000
max,1601.260000,71.000000,30.560000,103.000000,359.950000,40.500000


## Ridge

In [25]:
# define the model-- with default options
ridge=Ridge()
# fit (train) the model
ridge.fit(X_train, y_train)


Ridge()

In [26]:
print('ridge on train: {:.2%}'.format(ridge.score(X_train, y_train)))
print('ridge on test: {:.2%}'.format(ridge.score(X_test, y_test)))

ridge on train: 81.57%
ridge on test: 80.94%


In [27]:
ridge.coef_

array([ 1.75296482e+01, -3.43058113e+02, -8.87583227e-01, -2.10355016e-01,
        2.54866199e+00, -1.25092475e+02,  3.72851197e+02,  3.88588745e+02,
        3.95159418e+02,  3.11657727e+02,  1.84720318e+02,  5.90674456e+01,
       -7.34788372e+01, -1.91377626e+02, -1.97569161e+02, -1.73933333e+02,
       -1.27171591e+02, -1.56810322e+02, -1.44718351e+02, -1.40084336e+02,
       -1.33470922e+02, -1.24230081e+02, -1.25193383e+02, -1.20904274e+02,
       -1.19665672e+02, -1.13058498e+02, -3.60418818e+01,  1.23321803e+02,
        2.67434090e+02])

In [28]:
# Ridge(alpha=0) the same as regular linear regression

# samaller alpha-- more complex model-- higher chance of over-fitting
ridge1=Ridge(alpha=.01)

# somthing in between
ridge2=Ridge(alpha=1)

# larger alpha-- less complex (simpler) model-- lower chance of over-fitting and if we go too high with alpha,
# we might have under-fitting
ridge3=Ridge(alpha=10)

ridge1.fit(X_train, y_train)
ridge2.fit(X_train, y_train)
ridge3.fit(X_train, y_train)

Ridge(alpha=10)

In [29]:
# evaluation
print('ridge1 on train: {:.3%}'.format(ridge1.score(X_train, y_train)))
print('ridge1 on test: {:.3%}'.format(ridge1.score(X_test, y_test)),'\n')

print('ridge2 on train: {:.3%}'.format(ridge2.score(X_train, y_train)))
print('ridge2 on test: {:.3%}'.format(ridge2.score(X_test, y_test)),'\n')

print('ridge3 on train: {:.3%}'.format(ridge3.score(X_train, y_train)))
print('ridge3 on test: {:.3%}'.format(ridge3.score(X_test, y_test)),'\n')



ridge1 on train: 81.565%
ridge1 on test: 80.941% 

ridge2 on train: 81.565%
ridge2 on test: 80.941% 

ridge3 on train: 81.554%
ridge3 on test: 80.932% 



## Lasso

In [30]:
# define the model with default parameters
lasso=Lasso()
# train (fit) the model
lasso.fit(X_train, y_train)
# evaluating the model
print('lasso on train: {:.2%}'.format(lasso.score(X_train, y_train)))
print('lasso on test: {:.2%}'.format(lasso.score(X_test, y_test)))

lasso on train: 80.66%
lasso on test: 80.01%


In [31]:
# smaller alpha-- more complex model-- higher chance of over-fitting
lasso1=Lasso(alpha=.0001, max_iter=100000)

lasso2=Lasso(alpha=.001,max_iter=100000)

lasso3=Lasso(alpha=.1, max_iter=100000)

In [32]:
# larger alpha-- less complex (simpler) model-- higher chance of under-fitting
lasso4=Lasso(alpha=1)
lasso1.fit(X_train, y_train)
lasso2.fit(X_train, y_train)
lasso3.fit(X_train, y_train)
lasso4.fit(X_train, y_train)

Lasso(alpha=1)

In [33]:
print('lasso1 on train: {:.3%}'.format(lasso1.score(X_train, y_train)))
print('lasso1 on test: {:.3%}'.format(lasso1.score(X_test, y_test)))
print('lasso1 number of non-zero coeff:', sum(lasso1.coef_!=0),'\n')

print('lasso2 on train: {:.3%}'.format(lasso2.score(X_train, y_train)))
print('lasso2 on test: {:.3%}'.format(lasso2.score(X_test, y_test)))
print('lasso2 number of non-zero coeff:', sum(lasso2.coef_!=0),'\n')

print('lasso3 on train: {:.3%}'.format(lasso3.score(X_train, y_train)))
print('lasso3 on test: {:.3%}'.format(lasso3.score(X_test, y_test)))
print('lasso3 number of non-zero coeff:', sum(lasso3.coef_!=0),'\n')

print('lasso4 on train: {:.3%}'.format(lasso4.score(X_train, y_train)))
print('lasso4 on test: {:.3%}'.format(lasso4.score(X_test, y_test)))
print('lasso4 number of non-zero coeff:', sum(lasso4.coef_!=0),'\n')


lasso1 on train: 81.565%
lasso1 on test: 80.941%
lasso1 number of non-zero coeff: 28 

lasso2 on train: 81.565%
lasso2 on test: 80.941%
lasso2 number of non-zero coeff: 28 

lasso3 on train: 81.555%
lasso3 on test: 80.929%
lasso3 number of non-zero coeff: 27 

lasso4 on train: 80.656%
lasso4 on test: 80.010%
lasso4 number of non-zero coeff: 21 




## random forrest


In [34]:
from sklearn.ensemble import RandomForestRegressor

In [35]:
# define and fit the model
rf1=RandomForestRegressor(n_estimators=300, random_state=0)
rf1.fit(X_train, y_train)

RandomForestRegressor(n_estimators=300, random_state=0)

In [36]:
# evaluate
print('dt rf1 on train: {:.2%}'.format(rf1.score(X_train, y_train)))
print('dt rf1 on test: {:.2%}'.format(rf1.score(X_test, y_test)))

dt rf1 on train: 98.75%
dt rf1 on test: 90.46%


In [37]:
rf1.feature_importances_

array([6.03359906e-01, 2.90864472e-02, 5.21223282e-02, 3.61221340e-02,
       1.66049903e-02, 6.24585378e-06, 2.50009143e-02, 2.30219849e-02,
       2.23280108e-02, 1.78631134e-02, 1.81366060e-02, 1.87316232e-02,
       2.77303682e-02, 3.40907471e-02, 1.72254062e-02, 4.15319250e-03,
       1.62936005e-06, 4.82430422e-04, 2.32847629e-05, 1.66050243e-05,
       1.11408361e-05, 1.76476282e-06, 1.18490274e-08, 1.05803908e-08,
       1.62618332e-08, 3.42080134e-05, 8.07980021e-03, 2.09010406e-02,
       2.48640396e-02])

In [38]:
rf1_feat_imp=pd.DataFrame(data=rf1.feature_importances_,index=X_dummies.columns, columns=['importance'])
rf1_feat_imp.sort_values(by=['importance'],ascending=False)

,importance
Temperature,6.033599e-01
Humidity,5.212233e-02
WindDirection(Degrees),3.612213e-02
hour_17,3.409075e-02
Pressure,2.908645e-02
hour_16,2.773037e-02
hour_10,2.500091e-02
hour_9:,2.486404e-02
hour_11,2.302198e-02
hour_12,2.232801e-02


## Decision Tree for Radiation Data

In [39]:
train_scores, test_scores=validation_curve(DecisionTreeRegressor(random_state=0), X_train, y_train, param_name='max_depth',
                                           param_range=[2,3,4,5,6,7,8], cv=5)

In [40]:
train_scores.mean(axis=1)

array([0.58128074, 0.63594011, 0.68464214, 0.73238805, 0.76974895,
       0.80119294, 0.83357318])

In [41]:
test_scores.mean(axis=1)

array([0.57842823, 0.63043052, 0.67645032, 0.72622363, 0.7593272 ,
       0.78759558, 0.81526361])

In [42]:
dt_reg=DecisionTreeRegressor(random_state=0, max_depth=8)
dt_reg.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=8, random_state=0)

In [43]:
print('dt reg on train: {:.2%}'.format(dt_reg.score(X_train, y_train)))
print('dt reg on test: {:.2%}'.format(dt_reg.score(X_test, y_test)))

dt reg on train: 83.31%
dt reg on test: 81.27%


In [44]:
dt_reg.feature_importances_

array([0.71056434, 0.00508298, 0.02426736, 0.00745154, 0.00153916,
       0.        , 0.02767104, 0.02636303, 0.02317313, 0.01811013,
       0.01197649, 0.01426763, 0.0290787 , 0.03753499, 0.01875588,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01658492, 0.02757867])

In [45]:
X_solar.columns

Index(['Temperature', 'Pressure', 'Humidity', 'WindDirection(Degrees)',
       'Speed', 'hour'],
      dtype='object')

In [46]:
feat_imp=pd.DataFrame(data=dt_reg.feature_importances_, index=X_dummies.columns, columns=['importance'])
feat_imp.sort_values(by=['importance'], ascending=False)

,importance
Temperature,0.710564
hour_17,0.037535
hour_16,0.029079
hour_10,0.027671
hour_9:,0.027579
hour_11,0.026363
Humidity,0.024267
hour_12,0.023173
hour_18,0.018756
hour_13,0.018110


In [47]:
### visualization
export_graphviz(dt_reg,out_file='reg_tree1.dot', feature_names=X_dummies.columns, filled=True)

In [48]:
train_scores, test_scores=validation_curve(KNeighborsRegressor(), X_train, y_train, param_name='n_neighbors',
                                           param_range=[2,6,10,20,25], cv=5)

In [49]:
test_scores.mean(axis=1)

array([0.58527494, 0.65693275, 0.66552195, 0.66239797, 0.65870748])

In [51]:
NN=KNeighborsRegressor(n_neighbors=10)
NN.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [52]:
print('dt reg on train: {:.2%}'.format(NN.score(X_train, y_train)))
print('dt reg on test: {:.2%}'.format(NN.score(X_test, y_test)))

dt reg on train: 73.28%
dt reg on test: 66.27%


In [53]:
train_scores, test_scores=validation_curve(Ridge(), X_train, y_train, param_name='alpha',
                                           param_range=[.01,1,10], cv=5)

In [54]:
test_scores.mean(axis=1)

array([0.81517621, 0.81517461, 0.81500975])

In [55]:
rid=Ridge(alpha=.01)
rid.fit(X_train, y_train)

Ridge(alpha=0.01)

In [56]:
print('dt reg on train: {:.2%}'.format(rid.score(X_train, y_train)))
print('dt reg on test: {:.2%}'.format(rid.score(X_test, y_test)))

dt reg on train: 81.57%
dt reg on test: 80.94%


In [57]:
train_scores, test_scores=validation_curve(Lasso(max_iter=3000000), X_train, y_train, param_name='alpha',
                                           param_range=[.0001,.001,.1,1], cv=5)

In [58]:
test_scores.mean(axis=1)

array([0.8151762 , 0.81517619, 0.81507478, 0.80606956])

In [59]:
las=Lasso(alpha=.0001, max_iter=300000)
las.fit(X_train, y_train)

Lasso(alpha=0.0001, max_iter=300000)

In [60]:
print('dt reg on train: {:.2%}'.format(las.score(X_train, y_train)))
print('dt reg on test: {:.2%}'.format(las.score(X_test, y_test)))

dt reg on train: 81.57%
dt reg on test: 80.94%
